In [2]:
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 4))

## 10장 실습
### 10.1. 주간 평균 수익률 구하기
1. stock_px.csv 파일을 시계열 데이터 close_px 로 읽어라.
 - 날짜 데이터를 datetime으로 읽어야 한다.
 - 첫번재 컬럼을 index로 지정해야 한다.
2. shift 메소드를 이용하여 수익율을 계산하여 rate_px를 생성한다.
3. resample을 이용하여 주간 Resampler 를 wSampler를 생성한다.
4. wSampler를 이용하여 주간 평균 수익률을 계산한다.
5. 2000년 한 해 동안 주간 평균 수익률이 가장 높은 종목과 그 주간은?

In [1]:
path = 'C:/Users/tmznq/workspace/ml_scratch/seoul_coding_academy/pydata-book-1st-edition/ch11/stock_px.csv'

#### 1.1. 첫번째 컬럼을 index로 지정.

In [4]:
# close_px 데이터로 만들기
# index_col, parse_dates
close_px = pd.read_csv(path, index_col='Unnamed: 0', parse_dates=True)
close_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-01,4.98,7.86,2.87,16.79,4.27,0.51,6.04,328.79,6.12
1990-02-02,5.04,8.00,2.87,16.89,4.37,0.51,6.09,330.92,6.24
1990-02-05,5.07,8.18,2.87,17.32,4.34,0.51,6.05,331.85,6.25
1990-02-06,5.01,8.12,2.88,17.56,4.32,0.51,6.15,329.66,6.23
1990-02-07,5.04,7.77,2.91,17.93,4.38,0.51,6.17,333.75,6.33


In [5]:
close_px.index # dtype = 'object'

DatetimeIndex(['1990-02-01', '1990-02-02', '1990-02-05', '1990-02-06',
               '1990-02-07', '1990-02-08', '1990-02-09', '1990-02-12',
               '1990-02-13', '1990-02-14',
               ...
               '2011-10-03', '2011-10-04', '2011-10-05', '2011-10-06',
               '2011-10-07', '2011-10-10', '2011-10-11', '2011-10-12',
               '2011-10-13', '2011-10-14'],
              dtype='datetime64[ns]', length=5472, freq=None)

In [6]:
# from dateutil.parser import parse
# parse(data.index)

#### 1.2. index를 datetime으로 변환

In [7]:
close_px.index = pd.to_datetime(close_px.index)

In [8]:
close_px['2010']

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-01-04,16.40,214.01,14.66,128.32,60.84,29.76,58.10,1132.99,66.16
2010-01-05,15.89,214.38,14.74,126.77,60.14,29.77,58.80,1136.52,66.42
2010-01-06,16.72,210.97,14.66,125.94,60.63,29.58,58.21,1137.14,66.99
2010-01-07,16.36,210.58,15.42,125.51,60.19,29.28,57.84,1141.69,66.78
2010-01-08,16.77,211.98,15.75,126.77,60.40,29.48,57.65,1144.98,66.52
2010-01-11,17.19,210.11,15.90,125.44,60.41,29.10,57.59,1146.98,67.26
...,...,...,...,...,...,...,...,...,...
2010-12-23,15.25,323.60,17.60,144.04,60.62,27.78,64.20,1256.77,71.93
2010-12-27,15.14,324.68,17.75,143.50,60.31,27.55,63.94,1257.54,71.74
2010-12-28,15.16,325.47,17.87,143.86,60.43,27.49,63.80,1258.51,72.15


- 바뀐 거 확인

#### 2. shift 함수를 이용해서 수익률을 계산하여 rate_px 생성

In [9]:
rate_px = (close_px / close_px.shift(1) - 1).dropna()
rate_px.head()

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-02,0.012048,0.017812,0.000000,0.005956,0.023419,0.0,0.008278,0.006478,0.019608
1990-02-05,0.005952,0.022500,0.000000,0.025459,-0.006865,0.0,-0.006568,0.002810,0.001603
1990-02-06,-0.011834,-0.007335,0.003484,0.013857,-0.004608,0.0,0.016529,-0.006599,-0.003200
1990-02-07,0.005988,-0.043103,0.010417,0.021071,0.013889,0.0,0.003252,0.012407,0.016051
1990-02-08,0.000000,-0.007722,0.003436,-0.003904,0.018265,0.0,0.008104,-0.002367,0.003160


#### 3. wsampler 생성 

In [10]:
wsampler = rate_px.resample('W')
wsampler

DatetimeIndexResampler [freq=<Week: weekday=6>, axis=0, closed=right, label=right, convention=start, base=0]

#### 4. 주간 평균 수익률 계산

In [11]:
w_mean = wsampler.mean()
w_mean['2010']

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-01-03,-0.003357,0.002070,-0.004959,0.000679,-0.001102,-0.004165,-0.000681,-0.002525,-0.001703
2010-01-10,0.011482,0.001237,0.018833,-0.000019,-0.000600,0.001268,-0.000069,0.005318,0.003917
2010-01-17,-0.015555,-0.005719,-0.001873,0.001459,0.001104,0.001356,0.005015,-0.001541,-0.001183
2010-01-24,-0.037455,-0.009510,-0.004898,-0.011967,-0.005246,-0.015671,-0.007728,-0.009796,-0.011033
2010-01-31,-0.010109,-0.005429,-0.000357,-0.004954,-0.001039,-0.005292,-0.002555,-0.003274,-0.005073
2010-02-07,0.007902,0.003726,-0.003372,0.001917,-0.000663,-0.001018,-0.000240,-0.001296,0.001303
...,...,...,...,...,...,...,...,...,...
2010-11-21,-0.001299,-0.000740,-0.000301,0.001851,0.000532,-0.003273,0.000213,0.000137,-0.001175
2010-11-28,-0.003914,0.006784,-0.006376,-0.001887,-0.003894,-0.004213,-0.003094,-0.002101,-0.004623
2010-12-05,0.015704,0.001611,0.012090,0.002106,0.000851,0.013774,0.005477,0.005918,0.005662


#### 5. 

In [12]:
x = w_mean['2010']
x.max()

AA      0.022721
AAPL    0.018555
GE      0.018899
IBM     0.014687
JNJ     0.006371
MSFT    0.013774
PEP     0.007990
SPX     0.013327
XOM     0.010369
dtype: float64

In [13]:
w_mean

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-02-04,0.012048,0.017812,0.000000,0.005956,0.023419,0.000000,0.008278,0.006478,0.019608
1990-02-11,0.000815,0.000391,0.004837,0.010849,0.005481,0.003922,0.004906,0.001647,0.004153
1990-02-18,0.001686,-0.002216,-0.004077,0.001964,-0.000827,0.003846,-0.004148,-0.000516,-0.001087
1990-02-25,-0.001440,-0.003662,-0.006104,-0.002007,-0.015451,0.000605,-0.009518,-0.006491,-0.004348
1990-03-04,0.013098,0.003184,0.007732,0.004351,0.010759,0.014820,0.000411,0.006939,-0.001459
1990-03-11,-0.002971,0.017897,0.000716,0.003185,0.002326,0.003634,0.007103,0.001453,-0.001544
...,...,...,...,...,...,...,...,...,...
2011-09-11,-0.009252,0.002364,-0.010362,-0.008417,-0.001597,-0.000469,-0.013318,-0.004027,-0.003678
2011-09-18,0.006682,0.011918,0.017892,0.014026,0.002976,0.010519,0.006891,0.010492,0.009803
2011-09-25,-0.033765,0.002035,-0.013958,-0.004179,-0.009438,-0.015522,-0.005516,-0.013331,-0.014336


In [18]:
x[x == x.max()].dropna(how = 'all').fillna(' ')    

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-04-25,,0.0185547,,,,,,,
2010-05-16,,,,0.0146872,,,,,
2010-07-11,0.022721,,0.0188994,,,,0.00798992,0.0133266,
2010-08-08,,,,,0.00637051,,,,
2010-11-07,,,,,,,,,0.010369
2010-12-05,,,,,,0.0137736,,,


In [14]:
x[x == x.max()[:1]].dropna(how = 'all').fillna(' ')    

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-07-11,0.022721,,,,,,,,


In [15]:
w_mean.resample('Y').apply(lambda x: x[x == x.max()].dropna(how = 'all').fillna(' '))

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
1990-12-31,0.013098,0.029573,0.012062,0.014463,0.023419,0.019329,0.016688,0.008265,0.019608
1991-12-31,0.024316,0.025858,0.015930,0.017286,0.018609,0.033231,0.022045,0.012348,0.013119
1992-12-31,0.030883,0.025074,0.013240,0.014801,0.013723,0.025001,0.013073,0.007861,0.008177
1993-12-31,0.011470,0.048003,0.009303,0.018304,0.021472,0.017001,0.011628,0.004595,0.010273
1994-12-31,0.011449,0.025950,0.014889,0.021931,0.010088,0.016168,0.011806,0.006092,0.008894
1995-12-31,0.028171,0.023516,0.016908,0.013305,0.011714,0.020695,0.016989,0.005300,0.010121
...,...,...,...,...,...,...,...,...,...
2006-12-31,0.021094,0.037658,0.011634,0.011466,0.012291,0.014068,0.006937,0.007380,0.014387
2007-12-31,0.026322,0.022235,0.009689,0.013674,0.008757,0.030761,0.007923,0.007011,0.014345
2008-12-31,0.062814,0.031043,0.052032,0.026028,0.024973,0.025656,0.020137,0.029061,0.023495


In [16]:
# 참고

In [17]:
w_mean['2010'][w_mean['2010'] >= w_mean['2010'].max()[:1]]

,AA,AAPL,GE,IBM,JNJ,MSFT,PEP,SPX,XOM
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2010-11-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
